In [4]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())


2.10.0+cpu
CUDA available: False


In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1) load data
df = pd.read_csv("clean_oran_stage1.csv")

feature_cols = ["airtime", "selected_mcs"]
target_col = "pm_power"

df = df.dropna(subset=feature_cols + [target_col]).copy()
for c in feature_cols + [target_col]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=feature_cols + [target_col]).copy()

X = df[feature_cols].values
y = df[target_col].values

# 2) split: train/test then train/val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val,  y_train, y_val  = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# 3) scale (fit ONLY on train)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

print("Shapes:", X_train_s.shape, X_val_s.shape, X_test_s.shape)

Shapes: (12600, 2) (1400, 2) (3501, 2)


In [ ]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

class BaselineDNN(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x): return self.net(x)

def mean_relative_error(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return float(np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + eps)) * 100)

train_loader = DataLoader(TabularDataset(X_train_s, y_train), batch_size=64, shuffle=True)
val_loader   = DataLoader(TabularDataset(X_val_s, y_val), batch_size=64, shuffle=False)
test_loader  = DataLoader(TabularDataset(X_test_s, y_test), batch_size=64, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = BaselineDNN(in_dim=len(feature_cols)).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

best_val = float("inf")
best_state = None

for epoch in range(1, 101):
    # train
    model.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        opt.step()
        train_loss += loss.item() * len(xb)
    train_loss /= len(train_loader.dataset)

    # val
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            val_loss += loss_fn(pred, yb).item() * len(xb)
    val_loss /= len(val_loader.dataset)

    if val_loss < best_val:
        best_val = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | train MSE {train_loss:.6f} | val MSE {val_loss:.6f}")

# load best
model.load_state_dict(best_state)
print("Best val MSE:", best_val)


Epoch 001 | train MSE 42.701469 | val MSE 2.132022
Epoch 010 | train MSE 0.112595 | val MSE 0.109716
Epoch 020 | train MSE 0.114163 | val MSE 0.117217
Epoch 030 | train MSE 0.115693 | val MSE 0.181054
Epoch 040 | train MSE 0.117324 | val MSE 0.105616
Epoch 050 | train MSE 0.113591 | val MSE 0.105906
Epoch 060 | train MSE 0.114224 | val MSE 0.106978
Epoch 070 | train MSE 0.115933 | val MSE 0.106031
Epoch 080 | train MSE 0.115042 | val MSE 0.110421
Epoch 090 | train MSE 0.114863 | val MSE 0.111098
Epoch 100 | train MSE 0.115755 | val MSE 0.106632
Best val MSE: 0.10479746418339865


In [8]:
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        pred = model(xb).cpu().numpy().reshape(-1)
        y_pred.append(pred)
        y_true.append(yb.numpy().reshape(-1))

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

mse  = mean_squared_error(y_true, y_pred)
rmse = float(np.sqrt(mse))
mae  = mean_absolute_error(y_true, y_pred)
mre  = mean_relative_error(y_true, y_pred)

print("=== O-RAN Baseline DNN (A→A) ===")
print("X:", feature_cols, " y:", target_col)
print(f"MSE  : {mse:.6f}")
print(f"RMSE : {rmse:.6f}")
print(f"MAE  : {mae:.6f}")
print(f"MRE% : {mre:.4f}")


=== O-RAN Baseline DNN (A→A) ===
X: ['airtime', 'selected_mcs']  y: pm_power
MSE  : 0.106092
RMSE : 0.325717
MAE  : 0.249677
MRE% : 1.8607


In [9]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1) load data
df = pd.read_csv("clean_ul_stage1.csv")

feature_cols = ["airtime", "selected_mcs"]
target_col = "pm_power"

df = df.dropna(subset=feature_cols + [target_col]).copy()
for c in feature_cols + [target_col]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=feature_cols + [target_col]).copy()

X = df[feature_cols].values
y = df[target_col].values

# 2) split: train/test then train/val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val,  y_train, y_val  = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# 3) scale (fit ONLY on train)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

print("Shapes:", X_train_s.shape, X_val_s.shape, X_test_s.shape)

Shapes: (13614, 2) (1513, 2) (3782, 2)


In [10]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

class BaselineDNN(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x): return self.net(x)

def mean_relative_error(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return float(np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + eps)) * 100)

train_loader = DataLoader(TabularDataset(X_train_s, y_train), batch_size=64, shuffle=True)
val_loader   = DataLoader(TabularDataset(X_val_s, y_val), batch_size=64, shuffle=False)
test_loader  = DataLoader(TabularDataset(X_test_s, y_test), batch_size=64, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = BaselineDNN(in_dim=len(feature_cols)).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

best_val = float("inf")
best_state = None

for epoch in range(1, 101):
    # train
    model.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        opt.step()
        train_loss += loss.item() * len(xb)
    train_loss /= len(train_loader.dataset)

    # val
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            val_loss += loss_fn(pred, yb).item() * len(xb)
    val_loss /= len(val_loader.dataset)

    if val_loss < best_val:
        best_val = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | train MSE {train_loss:.6f} | val MSE {val_loss:.6f}")

# load best
model.load_state_dict(best_state)
print("Best val MSE:", best_val)


Epoch 001 | train MSE 609.473030 | val MSE 444.242185
Epoch 010 | train MSE 437.420263 | val MSE 436.842296
Epoch 020 | train MSE 437.028158 | val MSE 438.111650
Epoch 030 | train MSE 436.000050 | val MSE 436.971351
Epoch 040 | train MSE 435.124064 | val MSE 437.166793
Epoch 050 | train MSE 432.951646 | val MSE 433.744173
Epoch 060 | train MSE 428.263465 | val MSE 435.063481
Epoch 070 | train MSE 422.826160 | val MSE 423.910985
Epoch 080 | train MSE 420.164552 | val MSE 421.594999
Epoch 090 | train MSE 416.123310 | val MSE 425.029526
Epoch 100 | train MSE 412.566702 | val MSE 413.289641
Best val MSE: 412.8183861207426


In [11]:
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        pred = model(xb).cpu().numpy().reshape(-1)
        y_pred.append(pred)
        y_true.append(yb.numpy().reshape(-1))

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

mse  = mean_squared_error(y_true, y_pred)
rmse = float(np.sqrt(mse))
mae  = mean_absolute_error(y_true, y_pred)
mre  = mean_relative_error(y_true, y_pred)

print("=== O-RAN Baseline DNN (A→A) ===")
print("X:", feature_cols, " y:", target_col)
print(f"MSE  : {mse:.6f}")
print(f"RMSE : {rmse:.6f}")
print(f"MAE  : {mae:.6f}")
print(f"MRE% : {mre:.4f}")


=== O-RAN Baseline DNN (A→A) ===
X: ['airtime', 'selected_mcs']  y: pm_power
MSE  : 416.265656
RMSE : 20.402589
MAE  : 17.599899
MRE% : 90.6469
